In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn import metrics

In [ ]:
data = pd.read_csv('heart.csv')

msk_1 = np.random.rand(len(data)) < 0.85
dev = data[msk_1]
test = data[~msk_1]


In [ ]:
features_dev = dev.copy()
labels_dev = features_dev.pop('target').values

features_test = test.copy()
labels_test = features_test.pop('target').values

In [5]:
X_dev = features_dev.to_numpy()
y_dev = labels_dev
X_test = features_test.to_numpy()
y_test = labels_test

### Необходимые константы:

In [6]:
lr=0.0008
num_iter=600000
fit_intercept=True

### Функции обучения:

In [7]:
def add_intercept(X):
    intercept = np.ones((X.shape[0], 1))
    return np.concatenate((intercept, X), axis=1)

In [8]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [9]:
def fit(X, y):
    if fit_intercept:
        X = add_intercept(X)

In [10]:
def fit(X, y):
    if fit_intercept:
        X = add_intercept(X)

    # weights initialization
    theta = np.zeros(X.shape[1])

    for i in range(num_iter):
        z = np.dot(X, theta)
        h = sigmoid(z)
        gradient = np.dot(X.T, (h - y)) / y.size
        theta -= lr * gradient

        z = np.dot(X, theta)
        h = sigmoid(z)

    return theta

In [11]:
def predict_prob(X):
    if fit_intercept:
        X = add_intercept(X)

    return sigmoid(np.dot(X, theta))

### Процесс обучения модели:

In [12]:
theta = fit(X_dev, y_dev)

### Расчет True positive rate и False positive rate и площади под ROC-кривой для обучающей выборки:

In [13]:
y_score_proba_dev = predict_prob(X_dev)
y_true_dev = y_dev

In [14]:
fpr_dev, tpr_dev, thresholds_dev = metrics.roc_curve(y_true_dev, y_score_proba_dev)
auc = metrics.auc(fpr_dev,tpr_dev)

In [15]:
print(f"gini for dev sample: {round(100*((2 * auc) - 1), 4)}%")

gini for dev sample: 83.4129%


### Расчет True positive rate и False positive rate и площади под ROC-кривой для тестовой выборки:

In [16]:
y_score_proba_test = predict_prob(X_test)
y_true_test = y_test

In [17]:
fpr_test, tpr_test, thresholds_test = metrics.roc_curve(y_true_test, y_score_proba_test)
auc = metrics.auc(fpr_test,tpr_test)

In [18]:
print(f"gini for test sample: {round(100*((2 * auc) - 1), 4)}%")

gini for test sample: 89.0476%


### Расчет оптимального отсечения:

In [19]:
optimal_idx = np.argmax(tpr_dev - fpr_dev)
optimal_threshold = thresholds_dev[optimal_idx]
print(f'optimal cutoff for model: {optimal_threshold}')

optimal cutoff for model: 0.9999961695861664


### Расчет точности модели:

In [20]:
y_score_dev = [1 if predict >= optimal_threshold else 0 for predict in y_score_proba_dev]
y_score_test = [1 if predict >= optimal_threshold else 0 for predict in y_score_proba_test]

In [21]:
aimed_dev = 0
for s, t in zip(y_score_dev, y_true_dev):
    if s==t:
        aimed_dev += 1
        
aimed_test = 0
for s, t in zip(y_score_test, y_true_test):
    if s==t:
        aimed_test += 1

### Точность - отношение "угаданных" едениц класса к объему общей выборки:

In [22]:
Accuracy_dev = round(aimed_dev/len(y_dev), 4)
Accuracy_test = round(aimed_test/len(y_test), 4)

In [23]:
print(f'Точность обучающей выборки: {Accuracy_dev}\nТочность тестовой выборки: {Accuracy_test}')

Точность обучающей выборки: 0.8588
Точность тестовой выборки: 0.8537


In [24]:
test['my target'] = y_score_test

C:\Users\Nastya\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [25]:
test

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target,my target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1,1
7,44,1,1,120,263,0,1,173,0,0.0,2,0,3,1,1
8,52,1,2,172,199,1,1,162,0,0.5,2,0,3,1,1
16,58,0,2,120,340,0,1,172,0,0.0,2,0,2,1,1
23,61,1,2,150,243,1,1,137,1,1.0,1,0,2,1,0
26,59,1,2,150,212,1,1,157,0,1.6,2,0,2,1,1
44,39,1,2,140,321,0,0,182,0,0.0,2,0,2,1,1
47,47,1,2,138,257,0,0,156,0,0.0,2,0,2,1,1
57,45,1,0,115,260,0,0,185,0,0.0,2,0,2,1,1
62,52,1,3,118,186,0,0,190,0,0.0,1,0,1,1,1


### Найденные коеф. функции регрессии:

In [26]:
theta

array([ 1.07471839,  0.06272442, -5.42518818,  3.51479042, -0.07471172,
        0.01216357, -0.77308063,  1.37256805,  0.17443398, -3.98728052,
       -2.55575977,  2.30157256, -3.55725042, -4.44421202])

### Расчет для нового пациента:

In [ ]:
features_vector = []
for col_name in features_dev.columns:
    features_vector.append(float(input(f"Enter your {col_name}: ")))

Enter your age: 41


In [100]:
proba_vector = predict_prob(np.array([features_vector]))[0]

In [104]:
def answer(proba_vector, optimal_threshold):
    if proba_vector >= optimal_threshold:
        return 1
    else: 
        return 0

In [106]:
is_ill = answer(proba_vector, optimal_threshold)

In [107]:
if is_ill == 1:
    print("болен")
if is_ill == 0:
    print("здоров")

здоров
